## 15 搭建金融风控领域中的评分卡

### 15.1 理解金融领域中的风控问题
在风控领域，有一个很重要的问题是：如何通过用户的信息来判断用户的逾期与否？


所以，我们的问题是：基于用户相关的信息来预测此用户是否违约。

此项目中，我们采用的数据集是拍拍贷竞赛数据，链接为：   
“魔镜杯”风控算法大赛 https://www.kesci.com/home/competition/56cd5f02b89b5bd026cb39c9/content/1  


本项目中我们需要使用的模型是XGBoost, 请不要使用其他的模型。

在此项目中，我们仅仅使用Master表格里的数据，先忽略Loginfo和Userupdateinfo。

### 15.2 ROC与AUC

其实，任何的机器学习项目需要大量的预处理过程，本项目也不例外。  
真正想做好AI项目，我们必须要关注数据处理和特征方面的工作，这起着决定性的作用。模型固然重要，但其实没有想象中那么重要，因为很多都已经封装得特别好！  
但预处理和特征这块是需要动脑筋的，机器目前来讲是很难替代人工来完成这块的，所以这也是为什么很多之前的项目都需要花大量的时间来完成此部分。  

在本项目给定的数据表里，有大量不同种类的特征，而且也有缺失值。我们都需要仔细地去处理。此项目中可能需要涉及到的技术：

- 第一、**很多特征存在大量的缺失值**。那这些特征怎么办呢？ 删掉呢，还是保留呢？ 如果删掉，什么样的特征需要删掉呢？

对于此问题，我们可以设计一些规则。比如我们总共有100个样本，如果一个特征在超过60%以上的样本上都是NAN，可以删掉这个特征。具体的阈值可以通过可视化的方式来选择的。

- 第二、**样本本身具有大量的缺失值**。对于这个问题在第一部分也说过一次。一个简单的处理方式是：假如一个样本包含了超过50%以上的值为NAN，可以删掉此样本。


- 第三、在数据中有一些关于城市的数，而且城市本身对于逾期率还是有着很强的相关性。

比如某些地区的借款人，逾期率就会比其他地区的人高。所以这个特征本身是有价值的。但是除了头部几个城市跟逾期率有很大关系，剩下的许许多多城市倒是没有太大的关系。  
对于这样的特征我们可以采用二值化的处理。举个例子，如果我们发现城市A，城市B，城市C, 城市D的逾期率较高。那么，我们可以对这些城市最二值化的处理。  
具体做法如下: 比如一个样本的城市为”城市A“， 则通过二值化我们可以把城市信息表示为(1,0,0,0)，这里的每一个位置代表A,B,C,D其中一个。再比如一个样本的城市为C，我们则可以表示成(0,0,1,0)。如果一个样本的城市为”城市F“，由于城市F不属于这几个头部城市，所以直接就表示为(0,0,0,0)。针对于城市E也是一样的表示。  
通过这种转换，我们其实放大了几个特征的作用。在数据科学里是一种常见的手段。

- 第四、有些字符串的特征需要清洗。比如在一种特整理既出现“北京”，也出现“北京市”， 这时候就需要把“市”去掉，然后合并成同一个字符串。

- 第五、**可以设计一些衍生的特征**。举个例子，有两组特征，分别是户籍的所在地和申请贷款时的所在地。当申请贷款时的所在地不同于户籍所在地的时候就意味着这个人是在异地申请的，有可能存在一些风险。

对于这种情况，我们可以制造一个新的特征(随便取一个名字)。当户籍所在地和申请时的所在地一样的时候设置为0， 不一样的时候设置为1。

对于这类的延伸特征，我们还是需要理解数据的，不然很难凭空去想象。

- 第六、对于类别型特征，可以像往常一样用独热编码的形式来表示。

除了上述所谈到的这几个方面，当然你也可以去设计更多特征处理的方法。  
除了特征的预处理，在此项目中可以适当的做一下特征选择的工作。 建议使用**Tree-based Classifier**来选择特征，比如**决策树，XGBoost**等。  
用数据训练完这些模型之后，你可以通过模型自带的feature_importances_来获取每一个特征的重要性，然后通过排序即可以得到每个特征的重要性。  
之后基于选出来的特征重新构造训练数据和测试数据，之后再做最后的模型的训练。  

最后，简单说一下模型的评估。在此项目中，我们使用的评估方法叫做**AUC**(area under the curve)，是一种常用的用来评估二分类的评估标准。  
其实就是通过FP(false positive),FN(false negative),TP(true positive),TN(true negative)来画出一条**ROC曲线**，然后再计算它的面积，就可以得到AUC的值。  
使用方法请参考：https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score， 这个值越大模型越精准。如果AUC值为0.5，就意味着是一条对角线，意味着完全随机预测。

对于二分类问题，我们还是希望AUC能够达到0.8以上的，或者不要比这个值差太多。  
现阶段至少知道如何通过sklearn来计算AUC就可以了。

### 15.3 不平衡样本的处理
这节主要讲如何处理样本不平衡的问题。在本项目中，读取数据之后会发现正样本要比负样本少很多的。当然，如果正样本太多就意味着很多人都是逾期的。逾期的很多，那公司当然肯定运转不下去了嘛。  
假如正负样本差别比较大，比如20:1这种。可以采用适当的方法来处理，比如采样的方法。

- 举个例子，当正样本个数远远少于负样本的时候，我们可以采样更多的正样本。相当于，很多的正样本被我们重复使用，这个过程也叫作**over-sampling**。这样一来，至少训练的过程当中，正负样本比例差别不大的。


相反，如果负样本更少，我们则可以采样更多的负样本。


- 还有一种方式是，当正样本远远少于负样本时，我们可以部分采样负样本，使得采样后的负样本个数跟正样本差不多，这种过程叫作**under-sampling**。


上述两种方式是遇到样本不平衡问题时的常见的处理方式。 可以参考：  
[机器学习中样本不平衡的处理方法](https://zhuanlan.zhihu.com/p/28850865)